In [24]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import copy

def objective_f(x):
    f = x[0][0] ** 2 + (x[1][0] - 3) ** 2
    return f

def objective_df(x):
    df = np.array([[2 * x[0][0], 2 * (x[1][0] - 3)]])
    return df

def constraint_dg(x):
    dg = np.array([[-2, 2 * x[1][0]],
                   [5, 2 * (x[1][0] - 1)]])
    return dg

def constraint_g(x):
    g = np.array([[x[1][0] ** 2 - 2 * x[0][0]],
                  [(x[1][0] - 1) ** 2 + 5 * x[0][0] - 15]])
    return g

def line_search(x, s, lamda, w_old, k):  ## k number of iteration
    #
    t = 0.2
    a = 1
    if k == 0:
        w = abs(lamda)
    else:
        w = np.zeros((2, 1))
        w[0] = max(abs(lamda[0]), 0.5 * (w_old[0] + abs(lamda[0])))
        w[1] = max(abs(lamda[1]), 0.5 * (w_old[1] + abs(lamda[1])))

    dg_da_1 = 0 if constraint_g(x)[0, :] <= 0     else np.matmul(constraint_dg(x)[0, :], s)
    dg_da_2 = 0 if constraint_g(x)[1, :] <= 0 else np.matmul(constraint_dg(x)[1, :], s)
    df_da = np.matmul(objective_df(x), s) + (w[0, :] * dg_da_1 + w[1, :] * dg_da_2)

    def F_a(x, w, a, s):
        g1 = max(0, constraint_g(x + a*s)[0, :])
        g2 = max(0, constraint_g(x + a*s)[1, :])
        F = objective_f(x + a*s) + (w[0, :] * g1 + w[1, :] * g2)
        return F

    phi = lambda x, w, a, t, df_da: F_a(x, w, 0, 0) + a * t * df_da
    
    while phi(x, w, a, t, df_da) < F_a(x, w, a, s):
        a = 0.8 * a

    return a, w

In [25]:
def solve_sqp(x, W):
    A0 = constraint_dg(x)
    b0 = constraint_g(x)
    mu0 = np.zeros((b0.shape[0], 1))
    mu = []
    active = []
    while True:
        if len(active) == 0:
            matrix = W
            s_mu = np.matmul(np.linalg.inv(matrix), -objective_df(x).T)
            s = s_mu[:2, :]
            mu = []

        if len(active) != 0:
            if len(active) == 1:
                A = A0[active[0], :].reshape(1, -1)
                b = b0[active[0], :]
            if len(active) == 2:
                A = copy.deepcopy(A0)
                b = copy.deepcopy(b0)
            matrix = np.vstack((np.hstack((W, A.T)),
                                np.hstack((A, np.zeros((A.shape[0], A.shape[0]))))))
            s_mu = np.matmul(np.linalg.inv(matrix), np.vstack((-objective_df(x).T, -b)))
            s = s_mu[:2, :]
            mu = s_mu[2:, :]
            if len(mu) == 1:
                mu0[0] = s_mu[2:3, :]
            if len(mu) == 2:
                mu0[0] = s_mu[2:3, :]
                mu0[1] = s_mu[3:, :]

        sqp_constraint = np.round((np.matmul(A0, s.reshape(-1, 1)) + b0))

        mu_check = 0

        if len(mu) == 0:
            mu_check = 1
        elif min(mu) > 0:
            mu_check = 1
        else:
            id_mu = np.argmin(np.array(mu))
            mu.remove(min(mu))
            active.pop(id_mu)

        if np.max(sqp_constraint) <= 0:
            if mu_check == 1:
                return s, mu0
        else:
            index = np.argmax(sqp_constraint)
            active.append(index)
            active = np.unique(np.array(active)).tolist()